In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import datasets
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from qdrant_client import QdrantClient, models

tqdm.pandas()

In [3]:
client = QdrantClient(url="http://localhost:6333")

COLLECTION_NAME = "questions"

In [4]:
points = client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="rating",
                range=models.Range(gte=5),
            )
        ]
    ),
    limit=100_000,
    with_vectors=True,
    timeout=200,
)

In [5]:
vectors_sample = [i.vector for i in points[0]]

In [8]:
def create_prompt_dataset(tokenizer, system_prompt, question, answer, context):
    prompt = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": question,
        },
        {"role": "assistant", "content": answer},
    ]
    record = {}
    record["prompt"] = tokenizer.apply_chat_template(
                    prompt, tokenize=False, add_generation_prompt=False
                )
    record["prompt_not_answer"] = tokenizer.apply_chat_template(
                    prompt[:-1], tokenize=False, add_generation_prompt=True
                )
    record["answer"] = answer
    record["question"] = question
    
    if context:
        prompt = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": question,
        },
        {"role": "system", "content": context},
        {"role": "assistant", "content": answer},
    ]
        record["prompt_context"] = tokenizer.apply_chat_template(
                    prompt, tokenize=False, add_generation_prompt=False
                )
        record["prompt_context_not_answer"] = tokenizer.apply_chat_template(
                    prompt[:-1], tokenize=False, add_generation_prompt=True
                )
        record["context"] = context
    else:
        record["prompt_context"] = None
        record["prompt_context_not_answer"] = None
        record["context"] = None

    return record

model_name = "unsloth/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
system_prompt = (
    "Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, "
    "ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!"
)
question = "Хто я?"
answers = "Ты волшебник"
contexts = "Пример 1"

create_prompt_dataset(tokenizer, system_prompt, question, answers, contexts)

{'prompt': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>assistant\nТы ***<|im_end|>\n',
 'prompt_not_answer': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>assistant\n',
 'answer': 'Ты ***',
 'question': 'Хто я?',
 'prompt_context': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>system\nПример 1<|im_end|>\n<|im_start|>assistant\nТы ***<|im_end|>\n',
 'prom

In [9]:
system_prompt = (
    "Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, "
    "ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!"
)

payloads = []
for vector in tqdm(vectors_sample):
    vect_search = client.query_points(
        collection_name=COLLECTION_NAME, query=vector, limit=5
    ).points

    payload, vect_search = vect_search[0].payload, vect_search[1:]
    if payload["question"] == vect_search[0].payload["question"]:
        vect_search = vect_search[1:]
    else:
        vect_search = vect_search[:1]

    
    prav_sit = f"""Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\n"""
    for i in vect_search:
        prav_sit += f"""Вопрос гражданина:
    {i.payload["question"]}
    Ответ юриста:
    {i.payload["answer"]}
    """
    payloads.append({"system_prompt":system_prompt, "question":payload["question"], "answer":payload["answer"], "context":prav_sit})

 15%|█▍        | 13974/96234 [02:35<15:18, 89.60it/s]


KeyboardInterrupt: 

In [54]:
dataset = []

for record in tqdm(payloads):
    dataset.append(create_prompt_dataset(tokenizer, **record))

100%|██████████| 96234/96234 [00:23<00:00, 4134.79it/s]


In [57]:
dataset = pd.DataFrame(dataset)

In [65]:
dataset = dataset.sample(frac=1)

In [66]:
train_data, valid_data = dataset[15_000:], dataset[:15_000]

In [68]:
dataset = datasets.DatasetDict({"train": datasets.Dataset.from_pandas(train_data), "valid": datasets.Dataset.from_pandas(valid_data)})

In [77]:
dataset.save_to_disk("./data/processed/100k_rows_qwen/")

Saving the dataset (1/1 shards): 100%|██████████| 15000/15000 [00:00<00:00, 72817.10 examples/s]


In [94]:
print(dataset["train"]["prompt_context"][12])

<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
В 2018 году с супругом купили дом под маткапитал и в прошлом году мы развелись. В данный момент он проживает со мной и детьми, за все коммунальные услуги плачу я, у него задолжность по алиментам 300 тыс. Могу ли я как то подать на него в суд за неуплату коммунальных услуг?<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    В 2018 году с супругом купили дом под маткапитал и в прошлом году мы развелись. В данный момент он проживает со мной и детьми, за все коммунальные услуги плачу я, у него задолжность по алиментам 300 тыс. Могу ли я как то подать на него в суд за неуплату коммунальных услуг?
    Ответ юриста:
    Юлия, можете предъявить в суд исковые требования о взыскании 